In [1]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer, DPOConfig

c:\Users\Dell\Documents\llm_2024\llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [2]:
# Open and load the json dataset
with open("c:\\Users\\Dell\\Documents\\genAI_HW\\GenAI_hw6_dataset\\labelled_data.json", 'r', encoding='utf-8') as jsonfile:
    full_data = json.load(jsonfile)

with open("c:\\Users\\Dell\\Documents\\genAI_HW\\GenAI_hw6_dataset\\test_prompt.json", 'r', encoding='utf-8') as jsonfile:
    test_data = json.load(jsonfile)

Load model

In [3]:
# model_name = "NousResearch/Llama-2-7b-chat-hf"
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# tokenizer = AutoTokenizer.from_pretrained(model_name, token=cred.keys['HuggingFace_API_KEY'])
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    trust_remote_code=True,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_compute_dtype=torch.bfloat16,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type='nf4'
    # )
)

CUTOFF_LEN = 512  # Or 1024/2048 depending on your memory and use case

Get response from the original model

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt")
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('assistant')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


 10%|█         | 1/10 [00:42<06:23, 42.62s/it]

Response from original model:
|>
Yes, real-life characters can indeed improve the global reach and popularity of Japanese manga and anime. In fact, many Japanese manga and anime series have gained international popularity due to their realistic and relatable characters.

Real-life characters can help to create a more authentic and relatable story, which can appeal to a wider audience. This can be especially true for series that focus on everyday life or real-life situations, as these can resonate with people from all walks of life.

Additionally, real-life characters can help to build a sense of community and connection among fans. By sharing similar experiences and emotions, fans can form a strong bond and support each other through the ups and downs of the series.

Overall, real-life characters can be a powerful tool for improving the global reach and popularity of Japanese manga and anime. By creating characters that are relatable, authentic, and

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


 20%|██        | 2/10 [01:28<05:56, 44.55s/it]

...

Response from original model:
|>
Yes, real-life actors can indeed contribute to the development and diversification of Japanese manga and anime. In fact, real-life actors have been involved in the creation of many popular Japanese manga and anime series, including the popular "One Piece" series.

Real-life actors can bring a unique perspective and authenticity to the characters they portray, which can help to create a more immersive and engaging story for the audience. They can also help to bring the characters to life in a way that is more relatable and believable, which can help to increase the audience's engagement and enjoyment of the series.

Moreover, real-life actors can also help to promote the anime and manga industry in Japan by increasing awareness and interest in the medium. They can also help to attract new audiences to the industry by providing a glimpse into the world of anime and manga production

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [06:12<00:39, 39.19s/it]

Response from original model:
|>
Yes, real-life actors can be a significant factor in expanding the market for animated TV shows and movies. When fans see actors portraying their favorite characters, they are more likely to become fans of the show or movie. This can lead to increased sales and revenue for the show or movie, as fans are more likely to purchase merchandise, watch the show or movie again, and recommend it to others. Real-life actors can also help to establish a sense of authenticity and realism in animated TV shows and movies, which can appeal to a wider audience and increase the show's appeal to a broader demographic. Overall, real-life actors can be a valuable asset in expanding the market for animated TV shows and movies, helping to increase their popularity and revenue.

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [06:51<00:00, 41.11s/it]

Response from original model:
|>
Yes, realistic animation is considered to be a significant factor in enhancing the global competitiveness of Japanese manga and anime. The use of realistic animation techniques in Japanese animation has helped to create a more immersive and engaging viewing experience for audiences worldwide. This has led to increased popularity and recognition of Japanese anime and manga, particularly among younger audiences who are more likely to be drawn to realistic and detailed animation. Realistic animation has also helped to establish Japan as a leading animation production hub, with many of the world's most popular anime and manga series being produced in Japan. Overall, realistic animation has played a significant role in enhancing the global competitiveness of Japanese anime and manga, helping to establish Japan as a leading animation production hub and creating a more immersive and engaging viewing experience for audiences worldwide.



Set parameters

In [7]:
num_epoch = 1
data_size = 50
support_ratio = 1

Prepare training data

In [8]:
# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<|system|>\n回覆請少於20字</s>\n<|user|>\n日本動漫真人化是否有...,support,真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠擴大動...,support,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠吸引新...,support,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠保留原...,support,真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠提升動...,support,真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠保持原...,support,真人化可以透過場景、服裝等元素保留文化特色。,真人化可能因為文化差異而失去原作獨有的文化魅力。
6,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠挑戰技...,support,真人化促使技術創新，挑戰視覺效果上的新高度。,真人化可能因為技術限制而無法達到動畫中的視覺效果。
7,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否會受到演員...,support,演員選擇可因應市場需求，不必受限於動畫形象。,演員選擇可能引起爭議，觀眾難以接受角色塑造。
8,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠提高動...,support,真人化有機會讓更多人接受動漫，提高社會認同度。,真人化可能因為劇情改編而無法贏得社會認同。
9,<|system|>\n回覆請少於20字</s>\n<|user|>\n真人化是否能夠保留原...,support,真人化可以透過演員表現保留角色的個性特色。,真人化可能因演員演技或導演選擇而失去角色的原有特色。


Training

In [14]:
# training_args = TrainingArguments(
#     output_dir='./',
#     per_device_train_batch_size=1,
#     num_train_epochs=num_epoch,
#     gradient_accumulation_steps=8,
#     gradient_checkpointing=False,
#     learning_rate=2e-4,
#     optim="paged_adamw_8bit",
#     logging_steps = 1,
#     warmup_ratio = 0.1,
#     report_to = 'none'
# )

training_args = DPOConfig(
    output_dir="Qwen2-0.5B-DPO",
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="adamw_torch",
    logging_steps=10,
    warmup_ratio=0.1,
    report_to="none"
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    # beta=0.1,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

Tokenizing train dataset: 100%|██████████| 50/50 [00:00<00:00, 1302.12 examples/s]


In [15]:
dpo_trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=0.6615010499954224, metrics={'train_runtime': 195.5047, 'train_samples_per_second': 0.256, 'train_steps_per_second': 0.031, 'total_flos': 0.0, 'train_loss': 0.6615010499954224, 'epoch': 0.96})

Get response from the trained model

In [18]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt")
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('assistant')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


 10%|█         | 1/10 [00:41<06:16, 41.86s/it]

Response from trained model:
|>
Yes, real-life characters can indeed improve the global reach and popularity of Japanese manga and anime. In fact, many Japanese manga and anime series have gained international popularity due to their realistic and relatable characters.

Real-life characters can help to create a more authentic and relatable story, which can appeal to a wider audience. This can be especially important for manga and anime that are set in real-world settings, such as Japan or other countries.

Real-life characters can also help to build a sense of community and connection among fans of the series. This can be especially important for manga and anime that are popular among a younger audience, as they can help to create a sense of shared identity and cultural identity.

Overall, real-life characters can be a powerful tool for improving the global reach and popularity of Japanese manga and anime. By creating more relatable and authentic characters, these

Question 2:
真人化如何影響年

 20%|██        | 2/10 [01:22<05:31, 41.41s/it]

...

Response from trained model:
|>
Yes, real-life people can indeed contribute to the creative process of Japanese manga and anime. In fact, real-life people are often used as models or inspiration for characters in these works.

Real-life people can add depth and authenticity to the story, making it more relatable and engaging for readers. They can also help to break down stereotypes and create a more diverse and inclusive world for characters.

Moreover, real-life people can also help to promote the creative process by providing inspiration and feedback. They can offer insights into the challenges and opportunities that come with creating a manga or anime, and can help to shape the direction of the work.

Overall, real-life people can play a crucial role in the creative process of Japanese manga and anime, helping to bring the stories to life and creating a more diverse and inclusive world for readers.

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [06:05<00:39, 39.34s/it]

Response from trained model:
|>
Yes, real-life actors can be a significant factor in expanding the market for animated TV shows and movies. When fans see actors portraying their favorite characters, they are more likely to become fans of the show or movie. This can lead to increased sales and revenue for the show or movie, as fans are more likely to purchase merchandise, watch the show or movie, and attend events related to the show or movie. Real-life actors can also help to establish a connection between the show or movie and its fans, which can help to build brand loyalty and increase the show's overall popularity. Overall, real-life actors can be a valuable asset in expanding the market for animated TV shows and movies, helping to increase sales, revenue, and fan engagement.

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [06:45<00:00, 40.56s/it]

Response from trained model:
|>
Yes, realistic animation or "real-person animation" (RPA) is considered to be a significant factor in enhancing the global competitiveness of Japanese anime and manga. RPA is a technique that uses actors or animators to create a more realistic and lifelike appearance for characters in animated media.

RPA has been used in Japanese anime and manga for many years, and it has helped to establish Japan as a leading animation and manga production hub. RPA has been used to create characters that are more relatable, emotionally engaging, and visually appealing to audiences around the world.

For example, the popular anime series "Attack on Titan" (2013-present) features realistic and detailed character designs, which have helped to establish the series as a global phenomenon. The show's use of RPA has also contributed to its success, as it has



Please observe the output of this block to complete your report, and don't forget to take a screenshot of the results

In [19]:
model_response = []
print(f'num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}')
print()
for data in test_data:
    id = data['id']
    ref_output = original_model_response[id-1]
    output = trained_model_response[id-1]
    print(f'Question {id}:\n'+data['prompt'])
    print('Response from original model:\n'+ref_output)
    print('Response from trained model:\n'+output)
    print()
    model_response.append({'id':data['id'], 'prompt':data['prompt'], 'response_from_original_model':ref_output, 'response_from_trained_model':output})

num_epoch: 1
data_size: 50
support_ratio: 1

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
|>
Yes, real-life characters can indeed improve the global reach and popularity of Japanese manga and anime. In fact, many Japanese manga and anime series have gained international popularity due to their realistic and relatable characters.

Real-life characters can help to create a more authentic and relatable story, which can appeal to a wider audience. This can be especially true for series that focus on everyday life or real-life situations, as these can resonate with people from all walks of life.

Additionally, real-life characters can help to build a sense of community and connection among fans. By sharing similar experiences and emotions, fans can form a strong bond and support each other through the ups and downs of the series.

Overall, real-life characters can be a powerful tool for improving the global reach and popularity of Japanese manga and anime. By creating chara

Get the output file

In [ ]:
with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
    json.dump(model_response, outfile, indent=4, ensure_ascii=False)